# Age Vs Years of Code / pro. 

[Double stacked diagram](https://plotly.com/python/bar-charts/)

# Importar librerias

In [1]:
import pandas as pd
import plotly.express as px

import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv('../../data/survey_results_public2021.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta

#crear un dataframe con toda la informacion de la encuesta
df20 = pd.read_csv ('../../data/survey_results_public2020.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta

#pd.set_option("display.max_rows", None) #permite ver el output completo de la celda


# procesar datos. 

In [2]:
def ageCodePro (df):
    
    df1=df[['Age','YearsCode','YearsCodePro']].dropna().copy()

    #eliminar valores que no aportan informacion
    df1.drop(df1[df1['YearsCode'].isin(["Less than 1 year", "More than 50 years"])].index, inplace=True) 
    df1.drop(df1[df1['YearsCodePro'].isin(["Less than 1 year", "More than 50 years"])].index, inplace=True) 
    df1.drop(df1[df1['Age'].isin(["Prefer not to say"])].index, inplace=True) 

    #convertir columnas a entero
    df1['YearsCode'] = pd.to_numeric(df1['YearsCode'])
    df1['YearsCodePro'] = pd.to_numeric(df1['YearsCodePro'])
    
    if df1.dtypes['Age'] == 'float64': #si es el dataframe del anyo 2020, se lidiara con rangos:
        # etiquetas para la nueva columna
        labels = ["Under 18 years old", "18 - 24 years", "25 - 34 years",
                   "35 - 44 years", "45 - 54 years", "55 - 64 years", "65 - 74 years", 
                  "75 - 84 years", "Older than 84 years"]
        # divisiones de rango
        bins = [10, 17, 24, 34, 44, 54, 64, 74, 84, 100] 
        
        #eliminar valores de edad extremos.
        df1.drop(df1[(df1.Age < 10.0) | (df1.Age > 100.0)].index, inplace=True)
        
        # sobreescribir valores de columna usando las etiquetas
        df1['Age'] = pd.cut(df1['Age'], bins= bins, labels=labels)

        #agrupar por edad
    df2 = df1.groupby('Age', as_index = False).agg(median_code=('YearsCode', 'median'), 
                     median_pro=('YearsCodePro','median'), #nueva columna que calculara la medina de yearscodepro
                     # contar el numero de respuestas por edad                                                                                 
                     respuestas=('Age','count')) 

    return df2

In [3]:
# ageCodePro (df20) #ver el estado del df tras la funcion

# Layout 

In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}]
                )

app.layout = html.Div([

    dbc.Row([
            dbc.Col([

                html.H2('Double bar diagram'), 
                html.P('Comparison of different periods of development.'),
                dbc.Row([ # 
                    dbc.Col([
                        dbc.RadioItems( # dos botones con el anyo
                                id="opt1",
                                className="btn-group ml-auto",
                                inputClassName="btn-check",
                                labelClassName="btn btn-outline-primary",
                                labelCheckedClassName="active",
                                options=[
                                    {"label": "2021", "value": 2021},
                                    {"label": "2020", "value": 2020}
                                ],
                                value=2021,
                            ),
                    ]#, xs=5, sm=6, md=7, lg=8, xl=10
                    ) 
                ]),
                
                dcc.Graph(id='primero', figure={}, style={'height': '80vh'}),
            ]#, xs=5, sm=6, md=7, lg=8, xl=10
            )


        ], justify="center" 
        ), 

])


# Callback. 

In [5]:
@app.callback(
    Output(component_id='primero', component_property='figure'),
    Input(component_id='opt1', component_property='value'))
def update_graph(opt1):

    df = pd.DataFrame()
    
    if (opt1 == 2020):
        df = ageCodePro(df20)
    else:
        df = ageCodePro(df21)
        
    fig = px.bar(df, y="Age", x= ["median_pro", "median_code"],
             orientation = "h", barmode = 'group',
              text_auto= True)
        
    #poner el numero de la barra fuera de la barra y en posicion horizontal.
    fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
    fig['layout']['yaxis']['autorange'] = "reversed" #valores en orden descendente
    
    #renombrar ejes
    fig.update_xaxes(title_text="# Years")
    fig.update_yaxes(title_text="Age")
    fig.update_layout(title_text="Age Vs total of years development")
    
    #quitar color y grid del grafico
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
    fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
   

    return fig
#fin callback diagrama de barras dobles. 

# run

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
